In [2]:
from keras.layers import Conv2D, UpSampling2D, Dropout, Activation, InputLayer
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import tensorflow as tf

Using TensorFlow backend.
C:\Users\rajat\Anaconda3\envs\finalprojtf\lib\site-packages\matplotlib\colors.py:680: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  not cbook.is_string_like(colors[0]):


In [2]:
# Loading images
X = []
i = 0
for filename in os.listdir('test_images/Train3.1/'):
    if (i<1000):
        X.append(img_to_array(load_img('test_images/Train3.1/'+filename)))
        #os.remove('test_images/Train3.1/'+filename)
        i = i + 1
X = np.array(X, dtype=float)

In [3]:
# Splitting train and test sets
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128

In [4]:
# Model Structure
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 256)       590080    
__________

In [5]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [3]:
# Model Checkpointing 1 - loading
model = load_model('aivcudl.h5')
print("Model Loaded")

Model Loaded


In [7]:
# Train model      
%time model.fit_generator(image_a_b_gen(batch_size), epochs=5, steps_per_epoch=100)

Epoch 1/5
100/100 [==============================] - 260s - loss: 0.0098 - acc: 0.6731   
Epoch 2/5
100/100 [==============================] - 246s - loss: 0.0100 - acc: 0.6751   
Epoch 3/5
100/100 [==============================] - 246s - loss: 0.0097 - acc: 0.6829   
Epoch 4/5
100/100 [==============================] - 248s - loss: 0.0100 - acc: 0.6738   
Epoch 5/5
100/100 [==============================] - 249s - loss: 0.0096 - acc: 0.6850   
Wall time: 20min 51s


In [8]:
# Model checkpointing 2 - saving
model.save('aivcudl.h5')
print("Model Saved")

Model Saved


In [9]:
# Evaluation
print("Loss:\t\t\tAccuracy:")
print(model.evaluate(Xtest, Ytest, verbose=1, batch_size=batch_size))

Loss:			Accuracy:
50/50 [==============================] - 3s     
[0.013949483633041382, 0.58861571550369263]


In [12]:
# Batch image testing

color_me = []
for filename in os.listdir('test_images/Test2/'):
        color_me.append(img_to_array(load_img('test_images/Test2/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))
output = model.predict(color_me)
output = output * 128
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))

C:\Users\rajat\Anaconda3\envs\finalprojtf\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\rajat\Anaconda3\envs\finalprojtf\lib\site-packages\skimage\io\_io.py:132: UserWarning: result/img_20.png is a low contrast image
  warn('%s is a low contrast image' % fname)


In [4]:
# Single image Testing

color_me = load_img('pic1.jpg')
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,0]
color_me = color_me.reshape((1,)+color_me.shape+(1,))
output = model.predict(color_me)
output = output * 128
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("img_result.png", lab2rgb(cur))
    imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

C:\Users\rajat\Anaconda3\envs\finalprojtf\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\rajat\Anaconda3\envs\finalprojtf\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


In [8]:
# load json and create model(old load module)
from keras.models import model_from_json
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model")

Loaded model
